# Part 1 - Data Preprocessing

In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 71kB 5.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=e5ab0586d35892de9272f51f1f68dbfb30130237a34e3f4f2098cb4d4bb2f8c6
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=b3f97598648706effc0281441d42f52e62006dc28ef4f181eae200d32c123ff0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [29]:
# Importing the libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

In [30]:
# importing the data
data = pd.read_csv('/titanic_data.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
#---[ Check the Null values Column vice ]---
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [32]:
data = data.drop(['Cabin'],axis=1)
data = data.dropna()
#---[ Check the Null values Column vice ]---
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [33]:
x = data.drop(['PassengerId','Survived','Name','Ticket'],axis=1)
y = data['Survived']
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [34]:
sex = pd.get_dummies(x['Sex'])
embarked = pd.get_dummies(x['Embarked'])
x = pd.concat([x,sex,embarked],axis=1)
x = x.drop(['Sex','Embarked'],axis=1)
x

,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
885,3,39.0,0,5,29.1250,1,0,0,1,0
886,2,27.0,0,0,13.0000,0,1,0,0,1
887,1,19.0,0,0,30.0000,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,1,1,0,0


In [35]:
data_len,feature_size = x.shape
x.shape

(712, 10)

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Part 2 - Now let's make the ANN

In [39]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layer',min_value=2,max_value=15)):
        model.add(Dense(units=hp.Int('units '+str(i),
                                     min_value=5,
                                     max_value=15,
                                     step=2),
                        activation='relu',kernel_initializer='he_uniform'))
    #model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [40]:
tuner_model = RandomSearch(build_model,
                          objective = 'val_accuracy',
                          max_trials = 5,
                          executions_per_trial = 3,
                          directory='project3',
                          project_name = 'Titanic_data')
tuner_model.search_space_summary()

Search space summary
Default search space size: 3
num_layer (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 15, 'step': 1, 'sampling': None}
units 0 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 15, 'step': 2, 'sampling': None}
units 1 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 15, 'step': 2, 'sampling': None}


In [41]:
tuner_model.search(x_train,y_train,
             validation_data=(x_test,y_test),
             epochs=100)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.7834891080856323

Best val_accuracy So Far: 0.7834891080856323
Total elapsed time: 00h 03m 05s
INFO:tensorflow:Oracle triggered exit


In [42]:
tuner_model.results_summary()

Results summary
Results in project3/Titanic_data
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layer: 11
units 0: 11
units 1: 11
units 2: 13
units 3: 5
units 4: 11
units 5: 5
units 6: 13
units 7: 5
units 8: 11
units 9: 11
units 10: 9
units 11: 15
Score: 0.7834891080856323
Trial summary
Hyperparameters:
num_layer: 3
units 0: 7
units 1: 11
units 2: 7
units 3: 13
units 4: 7
units 5: 9
units 6: 13
units 7: 7
units 8: 7
units 9: 13
units 10: 13
units 11: 7
Score: 0.7834891080856323
Trial summary
Hyperparameters:
num_layer: 8
units 0: 9
units 1: 5
units 2: 13
units 3: 11
units 4: 7
units 5: 13
units 6: 13
units 7: 7
units 8: 13
Score: 0.7757009267807007
Trial summary
Hyperparameters:
num_layer: 12
units 0: 5
units 1: 9
units 2: 13
units 3: 15
units 4: 7
units 5: 9
units 6: 7
units 7: 9
units 8: 7
units 9: 5
units 10: 5
units 11: 5
Score: 0.7616822322209676
Trial summary
Hyperparameters:
num_layer: 9
units 0: 11
units 1: 5
units 2: 5